In [45]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import numpy as np

warnings.filterwarnings("ignore")

url = requests.get("https://finance.naver.com/sise/sise_market_sum.naver")
html = BeautifulSoup(url.text)

def make_table(html):
    table = html.find("table", class_ = 'type_2')
    table = pd.read_html(str(table))[0]
    del table['N']
    del table['토론실']
    table = table.dropna(subset = "종목명")
    return table

# 시가총액 - 체크박스
items = html.find('table', class_='item_list').find_all('input', attrs={'name': 'fieldIds'})

items_box = []
for i in items:
    items_box.append(i['value'])

In [35]:
# 코스피, 코스닥 모두 가져오고 (열도 모든 열) -> 하나의 데이터 프레임

result = []

# 코스닥
kosdaq_dic = {'menu' : 'market_sum',
              'returnUrl' : 'http://finance.naver.com/sise/sise_market_sum.naver?sosok=1&page=2',
              'fieldIds' : items_box}

url = requests.post('https://finance.naver.com/sise/field_submit.naver', data = kosdaq_dic)
html = BeautifulSoup(url.text)
result.append(make_table(html))



table = pd.concat([result[0], result[1]], ignore_index=True)

table.to_excel('시가총액.xlsx', index = False)
table

,종목명,현재가,전일비,등락률,액면가,거래량,거래대금,전일거래량,시가,고가,...,주당순이익,보통주배당금,매출액증가율,영업이익증가율,외국인비율,PER,ROE,ROA,PBR,유보율
0,피엔티,47350.0,상승 500,+1.07%,500.0,112137.0,5267.0,373217.0,46050.0,47700.0,...,3956.0,0.0,30.54,-1.08,5.83,11.97,20.01,5.09,2.23,3511.9
1,와이지엔터테인먼트,59200.0,상승 200,+0.34%,500.0,94756.0,5601.0,396710.0,58700.0,59700.0,...,134.0,300.0,45.51,86.39,12.83,441.79,14.00,10.58,2.38,4657.1
2,HK이노엔,37050.0,상승 350,+0.95%,500.0,55755.0,2056.0,176153.0,36750.0,37150.0,...,2030.0,350.0,-2.08,25.52,9.70,18.25,3.99,2.59,0.85,4308.0
3,유진테크,44800.0,하락 350,-0.78%,500.0,32733.0,1469.0,100394.0,46000.0,46000.0,...,1421.0,200.0,-10.98,-54.65,30.00,31.53,7.31,5.83,2.70,2946.6
4,디어유,42900.0,상승 250,+0.59%,500.0,189728.0,8151.0,439507.0,42650.0,43600.0,...,1048.0,NaN,53.86,75.77,3.29,40.94,16.86,15.26,5.38,1242.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,엔씨소프트,179200.0,"상승 5,400",+3.11%,500.0,33390.0,5914.0,67959.0,174100.0,179700.0,...,5768.0,3130.0,-30.79,-75.44,37.14,31.07,6.58,4.84,1.14,35584.0
96,넷마블,45500.0,상승 750,+1.68%,100.0,52508.0,2378.0,174315.0,44750.0,45700.0,...,228.0,0.0,-6.41,37.01,26.30,199.56,-4.90,-3.61,0.72,58501.8
97,현대건설,34950.0,상승 500,+1.45%,5000.0,539945.0,18992.0,3262793.0,34600.0,35850.0,...,3836.0,600.0,39.61,36.62,20.22,9.11,6.77,2.93,0.47,1337.2
98,현대차우,160000.0,"상승 1,500",+0.95%,5000.0,26041.0,4166.0,95241.0,159200.0,160700.0,...,45724.0,NaN,NaN,NaN,62.18,3.50,NaN,NaN,0.42,NaN


In [39]:
url = requests.get('https://finance.naver.com/sise/sise_market_sum.naver?sosok=0&page=1')
html = BeautifulSoup(url.text)
kospi_page = int(html.find('td', class_ = 'pgRR').find('a').get('href').split('=')[-1])

result = []

for n in tqdm(range(1, kospi_page + 1)):
    time.sleep(0.5 + random.random())
    # 코스피
    kospi_dic = {
        'menu' : 'market_sum',
        'returnUrl' : f'http://finance.naver.com/sise/sise_market_sum.naver?sosok=0&page={n}',
        'fieldIds' : items_box
    }
    
    url = requests.post('https://finance.naver.com/sise/field_submit.naver', data = kospi_dic)
    html = BeautifulSoup(url.text)
    result.append(make_table(html))

url = requests.get('https://finance.naver.com/sise/sise_market_sum.naver?sosok=1&page=1')
html = BeautifulSoup(url.text)
kosdaq_page = int(html.find('td', class_ = 'pgRR').find('a').get('href').split('=')[-1])

for n in tqdm(range(1, kosdaq_page + 1)):
    time.sleep(0.5 + random.random())    # 코스닥
    kospi_dic = {
        'menu' : 'market_sum',
        'returnUrl' : f'http://finance.naver.com/sise/sise_market_sum.naver?sosok=1&page={n}',
        'fieldIds' : items_box
    }
    
    url = requests.post('https://finance.naver.com/sise/field_submit.naver', data = kospi_dic)
    html = BeautifulSoup(url.text)
    result.append(make_table(html))



100%|███████████████████████████████████████████| 36/36 [00:48<00:00,  1.35s/it]


In [43]:
total = pd.concat([data for data in result], ignore_index = True)
total

,종목명,현재가,전일비,등락률,액면가,거래량,거래대금,전일거래량,시가,고가,...,주당순이익,보통주배당금,매출액증가율,영업이익증가율,외국인비율,PER,ROE,ROA,PBR,유보율
0,삼성전자,56600.0,하락 600,-1.05%,100.0,8999157.0,508452.0,14625181.0,57000.0,57100.0,...,4950.0,1446.0,16.20,398.34,50.07,11.43,9.03,7.10,0.98,41772.8
1,SK하이닉스,204500.0,"상승 4,000",+2.00%,5000.0,1607794.0,325994.0,3651613.0,199000.0,206000.0,...,14328.0,1200.0,-26.57,-213.52,55.84,14.27,-15.61,-8.95,2.16,1397.1
2,LG에너지솔루션,372500.0,"하락 1,000",-0.27%,500.0,64891.0,24132.0,140338.0,369500.0,376000.0,...,-1550.0,NaN,31.83,78.23,4.53,-240.32,6.36,3.91,4.31,16691.6
3,삼성바이오로직스,1159000.0,"하락 19,000",-1.61%,2500.0,26140.0,30356.0,108833.0,1181000.0,1187000.0,...,14788.0,NaN,23.10,13.22,13.58,78.37,9.12,5.26,7.79,5432.6
4,현대차,204500.0,"상승 2,000",+0.99%,5000.0,336261.0,69087.0,651155.0,203500.0,207000.0,...,45724.0,11400.0,14.43,53.96,37.18,4.47,13.68,4.56,0.54,6248.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4096,투비소프트,583.0,하락 28,-4.58%,500.0,192906.0,114.0,816914.0,611.0,629.0,...,-3132.0,0.0,-0.46,8.06,2.89,-0.19,-41.73,-17.86,0.20,-8.0
4097,셀리버리,116.0,하락 27,-18.88%,500.0,7268545.0,850.0,31418566.0,125.0,125.0,...,-166.0,0.0,-18.55,70.75,3.78,-0.70,472.60,-59.26,-0.21,-289.3
4098,CNH,109.0,보합0,0.00%,2500.0,0.0,0.0,0.0,0.0,0.0,...,-2570.0,0.0,-17.10,76.64,6.07,-0.04,-18.80,-3.89,0.36,5.8
4099,소프트센우,7060.0,상승 140,+2.02%,200.0,1186.0,8.0,204.0,7050.0,7060.0,...,-64.0,NaN,NaN,NaN,0.02,-110.31,NaN,NaN,13.72,NaN
